# Advent of Code 2020


In [60]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=7)

## Day1: 2SUM and 3SUM
In the part 1, find product of two numbers in a list such that the sum of the two numbers is 2020.

In [7]:
def day01_2sum(xs: list[int], sumval=2020) -> int:
    """2SUM in O(n)"""
    seen = set()
    for x in xs:
        if sumval - x in seen:
            return x * (sumval - x)
        seen.add(x)

    # not found
    return -1

In [8]:
with open("2020/day01_input.txt") as f:
    xs = [int(x) for x in f.readlines()]
    result = day01_2sum(xs)
    print(result)

786811


In the part 2, find product of three numbers in a list such that the sum of the two numbers is 2020.

In [9]:
def day01_3sum(xs: list[int], sumval=2020) -> int:    
    """3SUM in O(N^2)"""
    seen = set()
    n = len(xs)
    for i in range(n):
        x = xs[i]
        for j in range(i + 1, n):
            y = xs[j]
            z = sumval - x - y
            if z in seen:
                return x * y * z
            seen.add(y)
        seen.add(x)
    
    # not found
    return -1

In [10]:
with open("2020/day01_input.txt") as f:
    xs = [int(x) for x in f.readlines() if x.strip()]
    result = day01_3sum(xs)
    print(result)

199068980


## Day 2: Password validity

For example, `1-3 a` means the letter `a` can appear at least once, and at most three times. Assume each rule is about a single letter usage.

In [91]:
def day02_parse(line: str) -> tuple[int, int, str, str]:
    """
    >>> day02_parse("1-3 a: abcde")
    (1, 3, 'a', 'abcde')
    """
    rule, password = line.split(": ")
    range, letter = rule.split()
    from_, to_ = map(int, range.split("-"))
    return (from_, to_, letter, password)

def day02_isvalid(line: str) -> bool:
    """
    >>> day02_isvalid("1-3 a: abcde")
    True
    >>> day02_isvalid("1-3 b: cdefg")
    False
    >>> day02_isvalid("2-9 c: ccccccccc")
    True
    """
    from_, to_, letter, password = day02_parse(line)
    cnt = password.count(letter)
    return from_ <= cnt and cnt <= to_

In [92]:
with open("2020/day02_input.txt") as f:
    xs = [x for x in f.readlines() if x.strip()]
    result = sum(day02_isvalid(x) for x in xs)
    print(result)

607


In [93]:
def day02_isvalid_updated(line: str) -> bool:
    """
    >>> day02_isvalid_updated("1-3 a: abcde")
    True
    >>> day02_isvalid_updated("1-3 b: cdefg")
    False
    >>> day02_isvalid_updated("2-9 c: ccccccccc")
    False
    """
    from_, to_, letter, password = day02_parse(line)
    criteria1 = password[from_ - 1] == letter
    criteria2 = password[to_ - 1] == letter
    return criteria1 != criteria2

In [94]:
with open("2020/day02_input.txt") as f:
    xs = [x for x in f.readlines() if x.strip()]
    result = sum(day02_isvalid_updated(x) for x in xs)
    print(result)

321


## Day 3: Slope in 2D

In [95]:
def day03_slope(grid: list[str], hstepsize=3, vstepsize=1) -> int:
    """
    Grid is rectangular composed of cells with either '.' or '#'.
    """
    height = len(grid)
    width = len(grid[0])
    maxstep = (height + vstepsize - 1) // vstepsize
    s = "".join(grid[vstepsize * i][(hstepsize * i) % width] for i in range(1, maxstep))
    return s.count("#")

In [96]:
with open("2020/day03_input.txt") as f:
    grid = [line.strip() for line in f.readlines() if line.strip()]
    result = day03_slope(grid)
    print(result)

189


In [17]:
# Day 3 test 
grid = [
    "..##.......",
    "#...#...#..",
    ".#....#..#.",
    "..#.#...#.#",
    ".#...##..#.",
    "..#.##.....",
    ".#.#.#....#",
    ".#........#",
    "#.##...#...",
    "#...##....#",
    ".#..#...#.#",
]

In [18]:
with open("2020/day03_input.txt") as f:
    grid = [line.strip() for line in f.readlines() if line.strip()]
    move11 = day03_slope(grid, 1, 1)
    move31 = day03_slope(grid, 3, 1)
    move51 = day03_slope(grid, 5, 1)
    move71 = day03_slope(grid, 7, 1)
    move12 = day03_slope(grid, 1, 2)

    result = move11 * move31 * move51 * move71 * move12
    print(result)

1718180100


## Day 4: Passport scanning

In [19]:
from typing import Any, Union

def day04_has_fields(data: dict[str, str]) -> bool:
    """
    """
    required_fields = {'byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'}
    return all(f in data for f in required_fields)

def day04_parse(chunk: str) -> dict[str, str]:
    fields = [item.strip() for item in chunk.split() if item.strip()]
    d = dict()
    for field in fields:
        k, v = field.split(':')
        d[k] = v
    return d

In [20]:
with open("2020/day04_input.txt") as f:
    chunks = [chunk.strip() for chunk in f.read().split('\n\n')]
    ds = [day04_parse(chunk) for chunk in chunks]
    records_with_fields = [d for d in ds if day04_has_fields(d)]
    print(len(records_with_fields))

210


In [21]:
def day04_isvalid(x: dict[str, str]) -> bool:
    """
    byr (Birth Year) - four digits; at least 1920 and at most 2002.
    iyr (Issue Year) - four digits; at least 2010 and at most 2020.
    eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
    hgt (Height) - a number followed by either cm or in:
        If cm, the number must be at least 150 and at most 193.
        If in, the number must be at least 59 and at most 76.
    hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
    ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
    pid (Passport ID) - a nine-digit number, including leading zeroes.
    cid (Country ID) - ignored, missing or not.

    """
    if not day04_has_fields(x):
        return False
        
    byr = len(x["byr"]) == 4 and '1920' <= x["byr"] <= '2002'
    iyr = len(x["iyr"]) == 4 and '2010' <= x["iyr"] <= '2020'
    eyr = len(x["eyr"]) == 4 and '2020' <= x["eyr"] <= '2030'
    hgt = (
        (x["hgt"].endswith('cm') and '150' <= x["hgt"][:-2] <= '193') 
        or 
        (x["hgt"].endswith('in') and '59' <= x["hgt"][:-2] <= '76')
    )
    hcl = (
        x["hcl"].startswith('#') and len(x["hcl"][1:]) == 6 and 
        set(x["hcl"][1:]) < set('0123456789abcdef')
    )
    ecl = x["ecl"] in "amb blu brn gry grn hzl oth".split()
    pid = len(x["pid"]) == 9 and x["pid"].isdigit()

    return all([byr, iyr, eyr, hgt, hcl, ecl, pid])
    

In [22]:
result = sum(day04_isvalid(d) for d in records_with_fields)
print(result)

131


In [23]:
test = """
hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022
"""
rec = day04_parse(test)
print(rec)
day04_isvalid(rec)

{'hcl': '#888785', 'hgt': '164cm', 'byr': '2001', 'iyr': '2015', 'cid': '88', 'pid': '545766238', 'ecl': 'hzl', 'eyr': '2022'}


True

## Day 5: Flight seating

In the 7-letter seating code, the first `F`/`B` corresponds to `0`/`1` in the leftmost significant digit in the binary representation, and the second corresponds to the second leftmost significant digit, and so on. Similarly, `L`/`R` works in the exactly the same manner as `F`/`B`. Due to the clever choice of the seat ID, the ID itself is identical to the binary number translated from the sequence of `F`/`B`/`L`/`R` letters.

In [102]:
def day05_decode(code: str) -> int:
    """
    >>> day05_decode("FBFBBFF")
    44

    >>> day05_decode("RLR")
    5

    >>> day05_decode("FBFBBFFRLR")
    357
    """
    assert set(code) <= set("FBLR")
    table = str.maketrans(dict(F='0', B='1', L='0', R='1'))
    binseq = code.translate(table)
    return int(binseq, base=2)


In [26]:
with open("2020/day05_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    result = max(day05_decode(line) for line in lines)
    print(result)

963


Find the seat ID sandwitched by the existing seats in the list.

In [27]:
with open("2020/day05_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    seat_ids = [day05_decode(line) for line in lines]
    seat_ids.sort()
    for (a, b) in zip(seat_ids, seat_ids[1:]):
        if b - a == 2:
            print(a + 1)

592


## Day 6: union and intersection

A little bit boring.

In [28]:
def day06(lines: str, f=set.union) -> int:
    chunk = lines.strip()
    letters_set = [set(s.strip()) for s in chunk.split()]
    letters = f(*letters_set)
    return len(letters)

In [29]:
with open("2020/day06_input.txt") as f:
    chunks = [chunk.strip() for chunk in f.read().split("\n\n") if chunk.strip()]
    counts = [day06(chunk) for chunk in chunks]
    result = sum(counts)
    print(result)

6590


In [30]:
with open("2020/day06_input.txt") as f:
    chunks = [chunk.strip() for chunk in f.read().split("\n\n") if chunk.strip()]
    counts = [day06(chunk, set.intersection) for chunk in chunks]
    result = sum(counts)
    print(result)

3288


## Day 7: DAG

The bag relation may be represented by directed acyclic graph (DAG). A DAG may be represented by node -> (adjascent nodes). A node is represented by (tone, color).

In [31]:
import collections

Node = tuple[str, str]
Dag = dict[Node, collections.Counter[Node]]

def day07_parse(lines: list[str]) -> Dag:
    """Parse lines and get adjascency lists.
    """
    
    def parse(line: str) -> tuple[Node, collections.Counter[Node]]:
        words = line.split()
        node_orig: Node = (words[0], words[1])
        acc = collections.Counter() 
        if "contain no" in line:
            return (node_orig, acc)

        _, latter = line.split(" bags contain ")
        items = [xs.split() for xs in latter.split(", ")]
        for item in items:
            try:
                count = int(item[0])
                tone = item[1]
                color = item[2]
                node = (tone, color)
                acc.update({node: count})
            except ValueError:
                pass
        return (node_orig, acc)

    d = dict()
    for line in lines:
        node_orig, adj_nodes = parse(line)
        d[node_orig] = adj_nodes

    return d


def day07_part1(g: Dag, target: Node=("shiny", "gold")) -> int:
    """Find number of nodes reachable to the target"""

    def f(curr: Node) -> int:
        if g[curr]:
            if target in g[curr]:
                return g[curr][target]
            else:
                return max(count * f(node) for node, count in g[curr].items())   
        return 0

    return sum(f(node) > 0 for node in g)

In [32]:
test = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""

xs = [x.strip() for x in test.split("\n") if x.strip()]
print(xs)
g = day07_parse(xs)
day07_part1(g)

['light red bags contain 1 bright white bag, 2 muted yellow bags.', 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.', 'bright white bags contain 1 shiny gold bag.', 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.', 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.', 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.', 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.', 'faded blue bags contain no other bags.', 'dotted black bags contain no other bags.']


4

In [33]:
with open("2020/day07_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    g = day07_parse(lines)
    result = day07_part1(g)
    print(result)

254


In [34]:
import functools

def day07_part2(g: Dag, start: Node=("shiny", "gold")) -> int:
    """Find total number of edges in the downstream from the start node"""

    @functools.lru_cache()
    def f(curr: Node) -> int:
        if g[curr]:
            return sum(count * (1 + f(node)) for node, count in g[curr].items())
        return 0

    return f(start)

In [35]:
test = """
light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.
"""

xs = [x.strip() for x in test.split("\n") if x.strip()]
print(xs)
g = day07_parse(xs)
day07_part2(g)

['light red bags contain 1 bright white bag, 2 muted yellow bags.', 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.', 'bright white bags contain 1 shiny gold bag.', 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.', 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.', 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.', 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.', 'faded blue bags contain no other bags.', 'dotted black bags contain no other bags.']


32

In [36]:
with open("2020/day07_input.txt") as f:
    lines = [line.strip() for line in f.readlines() if line.strip()]
    g = day07_parse(lines)
    result = day07_part2(g)
    print(result)

6006


## Day 8: First assembly language

* `nop`
* `acc`
* `jmp`


In [37]:

def day08_part1(content: str) -> int:
    """
    string-input interface of assembly language execution.
    """
    tuples = [line.strip().split() for line in content.split("\n") if line.strip()]
    program = [(cmd, int(arg)) for cmd, arg in tuples]
    result, _ = day08_run(program)
    return result


def day08_run(program: list[tuple[str, int]]) -> tuple[int, bool]:
    """
    Execute assembly language composed of nop, jmp, and acc.
    """
    n = len(program)
    acc = 0
    i = 0
    seen = set()
    hist = []
    while i not in seen and i < n:
        seen.add(i)
        hist.append(i)
        cmd, arg = program[i]
        if cmd == "nop":
            i += 1
        elif cmd == "acc":
            i += 1
            acc += arg
        elif cmd == "jmp":
            i += arg
        else:
            raise ValueError("Unknown command: {}".format(cmd))

    is_terminated = i == n
    # print(hist)
    return acc, is_terminated


In [38]:
test = """
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
"""

day08_part1(test)

5

In [39]:
with open("2020/day08_input.txt") as f:
    result = day08_part1(f.read())
    print(result)

1610


### Day08: Part2
Replace single `jmp` with `nop` such that the program terminates. Here, program termination is meant by reaching to the last instruction (and finishing it).

The assembly program may be considered as a directed graph; a node is a tuple of head and tail indices, and edge is created by `jmp` operation. Each node has a single outgoing edge. Replacing `jmp` to `nop` corresponds to replacing an edge to another.

But the data contains merely 223 number of jumps hence naive search works.

In [40]:
from typing import Generator

AssemblyProgram = list[tuple[str, int]]

def day08_part2(content: str) -> int:
    """
    
    """
    def gen_jmp2nop(program: AssemblyProgram) -> Generator[AssemblyProgram, None, None]:
        n = len(program)
        for i in range(n):
            if program[i][0] == "jmp":
                new_program = program[:]
                new_program[i] = ("nop", program[i][1])
                yield new_program

    tuples = [line.strip().split() for line in content.split("\n") if line.strip()]
    program = [(cmd, int(arg)) for cmd, arg in tuples]
    for program_mod in gen_jmp2nop(program):
        acc, is_terminated = day08_run(program_mod)
        if is_terminated:
            return acc

    # failed
    return -10000000

In [41]:
with open("2020/day08_input.txt") as f:
    result = day08_part2(f.read())
    print(result)

1703


## Day 9: Sliding-window 2SUM

Given a sequence of integers, and we have a sliding window such that the immediately following number should be sum of two numbers (without replacement) in the window. The window size is 25 in part 1.

In [42]:
import itertools
from typing import Optional

def day09_part1(xs: list[int], window_size=25) -> Optional[int]:
    """Find the first integer that is NOT a 2-SUM from the immediately antecedent window."""
    def is_2sum(ys: list[int], target: int) -> bool:
        g = itertools.combinations(ys, 2)
        return any(sum(tup) == target for tup in g)

    n = len(xs)
    for i in range(window_size, n):
        window = xs[i - window_size : i]
        if not is_2sum(window, xs[i]):
            return xs[i]
    return None

In [43]:
content = """
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
"""

xs = [int(s) for s in content.split()]
result = day09_part1(xs, window_size=5)
print(result)

127


In [44]:
with open("2020/day09_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    result = day09_part1(xs, window_size=25)
    print(result)

22406676


Part 2 is to find a contiguous sequence whose sum equals to the answer from the Part 1. Here a contiguous sequence must contain at least two numbers. しゃくとり法でやってみる。

In [45]:
def day09_part2(xs: list[int], target=int) -> Optional[int]:
    """しゃくとり法"""
    n = len(xs)
    i = 0
    j = 2
    acc = sum(xs[i:j])
    while i < n - 2 and j < n:
        if acc == target:
            return min(xs[i : j]) + max(xs[i : j])
        elif acc < target:
            acc += xs[j]
            j += 1
        elif acc > target:
            acc -= xs[i]
            i += 1
        else:
            raise ValueError(f"Something is wrong: i={j}, j={j}")
    return None

In [46]:
content = """
35
20
15
25
47
40
62
55
65
95
102
117
150
182
127
219
299
277
309
576
"""

xs = [int(s) for s in content.split()]
result = day09_part2(xs, target=127)
print(result)

62


In [47]:
with open("2020/day09_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    result = day09_part2(xs, target=22406676)
    print(result)

2942387


## Day 10: First DP

In [48]:
import collections

def day10_diff_counts(xs: list[int]) -> collections.Counter[int]:
    """Return Counter of diference of sorted integers
    """
    ys = sorted(xs)
    ys.insert(0, 0)        # starts from 0
    ys.append(ys[-1] + 3)  # ends with the max + 3
    counter = collections.Counter(b - a for a, b in zip(ys, ys[1:]))
    return counter


def day10_part1(xs: list[int]) -> int:
    """Return the count of diff==1 times the one of diff==3"""
    counter = day10_diff_counts(xs)
    print(counter)
    return counter[1] * counter[3]

In [49]:
test = """
16
10
15
5
1
11
7
19
6
12
4
"""
xs = [int(s) for s in test.split()]
day10_diff_counts(xs)

Counter({1: 7, 3: 5})

In [50]:
with open("2020/day10_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    result = day10_part1(xs)
    print(result)

Counter({1: 70, 3: 33})
2310


In [51]:
def day10_part2(xs: list[int]) -> int:
    """Return number of steps
    """
    ys = sorted(xs)
    ys.insert(0, 0)        # starts from 0
    ys.append(ys[-1] + 3)  # ends with the max + 3

    dp = [0 for _ in ys]
    dp[0] = 1
    n = len(dp)
    for i in range(1, n):
        k = i - 1
        while k >= 0 and ys[i] - ys[k] <= 3:
            dp[i] += dp[k]
            k -= 1
            
    return dp[-1]



In [52]:
test = """
28
33
18
42
31
14
46
20
48
47
24
23
49
45
19
38
39
11
1
32
25
35
8
17
7
9
4
2
34
10
3
"""
xs = [int(s) for s in test.split()]
day10_part2(xs)

19208

In [53]:
with open("2020/day10_input.txt") as f:
    xs = [int(s) for s in f.readlines() if s.strip()]
    result = day10_part2(xs)
    print(result)

64793042714624


## Day 11: 2-D Automata

In Part 1, find the number of iterations till the state reaches a fixed point.

In [54]:
Grid = list[list[str]]

def day11_map(xs: Grid) -> Grid:
    """Evolve state in 2-D grid by one step according to the rule.

    - If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
    - If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.

    This is naive implementation. Introduce convoluiton if performance matters.
    """
    horiz = len(xs[0])
    vert = len(xs)
    diffs = (-1, 0, +1)

    def is_valid(i, j, dx, dy):
        return (dx, dy) != (0, 0) and 0 <= j + dx < horiz and 0 <= i + dy < vert

    zs = [['.' for _ in range(horiz)] for _ in range(vert)]

    for i in range(vert):
        for j in range(horiz):
            if xs[i][j] == 'L':
                criteria = all(
                    xs[i + dy][j + dx] != '#' for dx in diffs for dy in diffs 
                    if is_valid(i, j, dx, dy)
                )
                zs[i][j] = '#' if criteria else 'L'
            elif xs[i][j] == '#':
                neighbors = sum(
                    1 for dx in diffs for dy in diffs 
                    if is_valid(i, j, dx, dy) and xs[i + dy][j + dx] == '#')
                zs[i][j] = 'L' if neighbors >= 4 else '#'
            else:
                assert xs[i][j] == '.'
                zs[i][j] = xs[i][j]
    
    return zs

def serialize(grid: Grid) -> str:
    return "\n".join("".join(line) for line in grid)

def day11_part1(state: Grid) -> int:
    """Count number of # in a fixed point."""
    ...
    prev = None
    cnt = 0
    while prev != state:
        cnt += 1
        prev = state = state
        state = day11_map(state)

    print(f"count: {cnt}")
    # print(serialize(state))

    result = sum(1 for xs in state for x in xs if x == '#')
    return result

In [55]:
s = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

grid = [list(line.strip()) for line in s.split() if line.strip()]
day11_part1(grid)

count: 6


37

In [56]:
with open("2020/day11_input.txt") as f:
    grid = [list(line.strip()) for line in f.readlines() if line.strip()]
    result = day11_part1(grid)
    print(result)

count: 122
2296


#### Day 11: Part 2
Observation of the rule gives us that a cell with `.` is permanent, and `#` and `L` can flip to the other. So, we focus on coordinates of `#` and `L`, and see how occupied coordnates (depicted as `#`) change in each step. Another observation tells that each coordinate point always refers the same neighbors in the eight directions.

So we pre-compute neighbors for each cell (as `dict[Coordinate, set[Coordinate]]`) before iterative application of the rules. We also set "state" by `dict[Coordinate, bool]` where the dictionary holds seat coordinates as its keys, and occupied status as its values.

In [57]:
from typing import Iterable
import collections

Coordinate = tuple[int, int]
Grid = list[list[str]]


def day11_seats(xs: Grid) -> dict[Coordinate, bool]:
    horiz = len(xs[0])
    vert = len(xs)
    result = {
        (i, j): xs[i][j] == '#' 
        for i in range(vert) for j in range(horiz) if xs[i][j] != '.'
    }
    return result

def day11_neighbors(seats: Iterable[Coordinate]) -> dict[Coordinate, set[Coordinate]]:
    neighbors = collections.defaultdict(set)

    def scan_lines(d: dict[int, list[Coordinate]]):
        for _, coodinates in d.items():
            line = sorted(coodinates)
            for coord, coord_next in zip(line, line[1:]):
                neighbors[coord].add(coord_next)
                neighbors[coord_next].add(coord)

    # prepare scanning diagonally/antidiagonally
    rows = collections.defaultdict(list)
    cols = collections.defaultdict(list)
    diags = collections.defaultdict(list)
    anti_diags = collections.defaultdict(list)
    for (i, j) in seats:
        rows[i].append((i, j))
        cols[j].append((i, j))
        diags[i - j].append((i, j))
        anti_diags[i + j].append((i, j))

    # scan horizontally, vertically, diagonally, anti-diagonally
    scan_lines(rows)
    scan_lines(cols)
    scan_lines(diags)
    scan_lines(anti_diags)

    return dict(neighbors)      


def day11_part2_update(seats: dict[Coordinate, bool], neighbors: dict[Coordinate, set[Coordinate]]) -> bool:
    """Update `seats` by modifying it. Also returns if the state is chaning."""
    diffs = dict()
    for s, is_occupied in seats.items():
        if is_occupied:
            if sum(1 for t in neighbors[s] if seats[t]) >= 5:
                diffs[s] = False
        else:
            # when the seat is vacant
            if all(seats[t] is False for t in neighbors[s]):
                diffs[s] = True

    is_changing = bool(diffs)
    for k, v in diffs.items():
        seats[k] = v

    return is_changing
    

def day11_part2(xs: Grid) -> int:
    seats = day11_seats(xs)
    neighbors = day11_neighbors(seats.keys())
    is_changing = True
    cnt = 0
    
    while is_changing:
        is_changing = day11_part2_update(seats, neighbors)
        if is_changing:
            cnt += 1
    
    print(f"{cnt = }")
    return sum(seats.values())


def day11_display(seats: dict[Coordinate, bool], shape: tuple[int, int]):
    v, h = shape
    grid = [['.' for _ in range(h)] for _ in range(v)]
    for i in range(v):
        for j in range(h):
            if (i, j) in seats:
                grid[i][j] = '#' if seats[(i, j)] else 'L'
    
    s = "\n".join("".join(line) for line in grid)
    print(s)


In [58]:
s = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""

grid = [list(line.strip()) for line in s.split() if line.strip()]
day11_part2(grid)

cnt = 6


26

In [59]:
seats = day11_seats(grid)
neighbors = day11_neighbors(seats.keys())
shape = len(grid), len(grid[0])
day11_display(seats, shape)

L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL


In [60]:
day11_part2_update(seats, neighbors)
day11_display(seats, shape)

#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##


In [61]:
day11_part2_update(seats, neighbors)
day11_display(seats, shape)

#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#


In [62]:
with open("2020/day11_input.txt") as f:
    grid = [list(line.strip()) for line in f.readlines() if line.strip()]
    result = day11_part2(grid)
    print(result)

cnt = 87
2089


## Day 12: 2-D grid movement

In [63]:
def day12_parse(content: str) -> list[tuple[str, int]]:
    lines = [line.strip() for line in content.split() if line.strip()]
    res = [(line[0], int(line[1:])) for line in lines]
    return res

def day12_map(state: tuple[int, int, int], cmd: tuple[str, int]) -> tuple[int, int, int]:
    """trajectory"""
    DIRS = "ESWN"

    x, y, dir_idx = state
    dir_ = DIRS[dir_idx]
    cmd_name, cmd_value = cmd
    if cmd_name == "E" or (cmd_name == "F" and dir_ == "E"):
        x += cmd_value
    elif cmd_name == "W" or (cmd_name == "F" and dir_ == "W"):
        x -= cmd_value
    elif cmd_name == "S" or (cmd_name == "F" and dir_ == "S"):
        y += cmd_value
    elif cmd_name == "N" or (cmd_name == "F" and dir_ == "N"):
        y -= cmd_value
    else:
        assert cmd_name in "RL"
        assert dir_idx in [0, 1, 2, 3]
        assert cmd_value % 90 == 0
        if cmd_name == "R":
            dir_idx = (dir_idx + cmd_value // 90) % 4
        else:
            dir_idx = (dir_idx - cmd_value // 90) % 4

    return (x, y, dir_idx)


def day12_part1(commands: list[tuple[str, int]]) -> int:
    state = (0, 0, 0)
    for cmd in commands:
        state = day12_map(state, cmd)
    x, y, _ = state

    return abs(x) + abs(y)



In [64]:
content = """
F10
N3
F7
R90
F11
"""
commands = day12_parse(content)
result = day12_part1(commands)
print(result)

25


In [65]:
with open("2020/day12_input.txt") as f:
    commands = day12_parse(f.read())
    result = day12_part1(commands)
    print(result)

2057


In [66]:
Coordinate = tuple[int, int]
def day12_map2(state: Coordinate, waypoint: Coordinate, cmd: tuple[str, int]) -> tuple[Coordinate, Coordinate]:
    """Now let E as +x, N as +y"""
    x, y = state
    wpx, wpy = waypoint

    cmd_name, cmd_value = cmd
    if cmd_name == "F":
        x += cmd_value * wpx
        y += cmd_value * wpy
    elif cmd_name == "E":
        wpx += cmd_value
    elif cmd_name == "W":
        wpx -= cmd_value
    elif cmd_name == "N":
        wpy += cmd_value
    elif cmd_name == "S":
        wpy -= cmd_value
    else:
        assert cmd_name in "RL"
        assert cmd_value % 90 == 0
        if (cmd_name, cmd_value) in [("R", 90), ("L", 270)]:
            wpx, wpy = wpy, -wpx
        elif (cmd_name, cmd_value) in [("L", 90), ("R", 270)]:
            wpx, wpy = -wpy, wpx
        elif cmd_value == 180:
            wpx, wpy = -wpx, -wpy

    return (x, y), (wpx, wpy)


def day12_part2(commands: list[tuple[str, int]]) -> int:
    state = (0, 0)
    waypoint = (10, 1)
    for cmd in commands:
        state, waypoint = day12_map2(state, waypoint, cmd)
        # print(f"{state = }")
        # print(f"{waypoint = }")
    x, y = state

    return abs(x) + abs(y)

In [67]:
content = """
F10
N3
F7
R90
F11
"""
commands = day12_parse(content)
result = day12_part2(commands)
print(result)

286


In [68]:
with open("2020/day12_input.txt") as f:
    commands = day12_parse(f.read())
    result = day12_part2(commands)
    print(result)

71504


## Day 13: Chinese remainder theorem

Or, extended Euclidean algorithm.

In [69]:
def day13_parse(content: str) -> tuple[int, list[int]]:
    lines = [line.strip() for line in content.split() if line.strip()]
    t0 = int(lines[0])
    print(lines[1])
    xs = [int(s) for s in lines[1].strip().split(",") if s.strip() and s != "x"]
    return t0, xs


def day13_part1(t0: int, xs: list[int]) -> int:
    def waittime(n: int, divisor: int):
        m = (divisor - n % divisor) % divisor
        return m

    tmp = [(waittime(t0, x), x) for x in xs]
    print(tmp)
    wait, x_selcted = min(tmp)
    print(f"{wait = }")
    print(f"{x_selcted = }")
    result = wait * x_selcted
    return result

In [70]:
content = """
939
7,13,x,x,59,x,31,19
"""

t0, xs = day13_parse(content)
print(f"{xs = }")
day13_part1(t0, xs)

7,13,x,x,59,x,31,19
xs = [7, 13, 59, 31, 19]
[(6, 7), (10, 13), (5, 59), (22, 31), (11, 19)]
wait = 5
x_selcted = 59


295

In [71]:
with open("2020/day13_input.txt") as f:
    t0, xs = day13_parse(f.read())
    result = day13_part1(t0, xs)
    print(result)

19,x,x,x,x,x,x,x,x,x,x,x,x,37,x,x,x,x,x,751,x,29,x,x,x,x,x,x,x,x,x,x,13,x,x,x,x,x,x,x,x,x,23,x,x,x,x,x,x,x,431,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,17
[(14, 19), (11, 37), (7, 751), (10, 29), (8, 13), (15, 23), (359, 431), (36, 41), (14, 17)]
wait = 7
x_selcted = 751
5257


In [49]:
from typing import Optional, Iterable
import operator
import functools
import math
import sympy


def modinv(a: int, p: int) -> Optional[int]:
    """Comptue a solution to modular multiplicative inverse x such that
    
    x = a^{-1} (mod p)
    
    or
    
    a x = 1 (mod p)
    
    when a and p are coprime. Otherwise, solution does not exist.
    Note that a solution is picked up from the class of solutions.
    Use extended Euclidean algorithm.
    
    >>> modinv(5, 19)
    4
    
    >>> modinv(21, 14) is None
    True
    
    >>> modinv(25, 14)
    9
    """
    d, x, y = extended_euclid(a, p)
    if d > 1:
        return None

    if x < 0:
        x %= p
    return x


def extended_euclid(a: int, b: int) -> tuple[int, int, int]:
    """Extended Eulidean algorithm returning the triple
    (d, x, y) satisfying d = a*x + b*y.

    Note that

    d = a*x + b*y 
      = b*x' + (a - b*(a//b))*y'
      = a*y' + b*(x' - (a//b)*y')

    So, chosing following gets 
    x = y'
    y = x' - (a//b)*y'

    >>> extended_euclid(5, 19)
    (1, 4, -1)

    >>> extended_euclid(240, 46)
    (2, -9, 47)
    """
    if a < b:
        d, y, x = extended_euclid(b, a)
        return d, x, y

    if b == 0:
        return (a, 1, 0)
    
    d, x_next, y_next = extended_euclid(b, a % b)
    x, y = y_next, x_next - (a // b) * y_next
    return (d, x, y)


def chinese_remainder(xs: list[tuple[int, int]]) -> Optional[int]:
    """Chinese remainder theorem

    For [(a_0, p_0), (a_1, p_1), ...], find x satisfying equations.

    x = a_i (mod p_i)

    >>> chinese_remainder([(2, 3), (3, 5), (2, 7)])
    23

    >>> chinese_remainder([(23, 84), (7, 160), (2, 63)])
    9767
    """
    # make divisors pairwise coprime
    qs = dict()
    for (_, p) in xs:
        for (k, n) in sympy.ntheory.factorint(p).items():
            if k in qs and qs[k] >= n:
                continue
            qs[k] = n

    factors = [p ** n for (p, n) in qs.items()]
    # print(f"{factors = }")

    mod_rem_pairs = dict()
    for k in factors:
        for (a, p) in xs:
            if p % k == 0:
                mod_rem_pairs[k] = a % k
    
    xs = [(r, p) for p, r in mod_rem_pairs.items()]
    # print(f"{xs = }")

    series = []
    p_product = math.prod(p for (_, p) in xs)
    # print(f"{p_product = }")
    for (a, p) in xs:
        m = p_product // p
        minv = modinv(m, p)
        if minv is None:
            raise ValueError(f"Unsupported operation: modinv({m}, {p})")
        series.append(a * m * minv)
    
    res = sum(series) % p_product
    return res


def day13_part2(content: str) -> Optional[int]:
    """Find timestamp t in Day 13 part 2 using Chinese remainder theorem. t0 is disregarded."""
    lines = [line.strip() for line in content.split() if line.strip()]
    xs = [(i, int(s)) for (i, s) in enumerate(lines[1].split(",")) if s != "x"]
    xs_fixed = [(-a % p, p) for (a, p) in xs]
    t = chinese_remainder(xs_fixed)
    return t


In [50]:
modinv(5, 19)

4

In [51]:
extended_euclid(5, 19)

(1, 4, -1)

In [52]:
modinv(21, 14)

In [53]:
chinese_remainder([(2, 3), (3, 5), (2, 7)])

23

In [54]:
chinese_remainder([(23, 84), (7, 160), (2, 63)])

9767

In [55]:
test = "123\n7,13,x,x,59,x,31,19"
day13_part2(test)

1068781

In [56]:
with open("2020/day13_input.txt") as f:
    result = day13_part2(f.read())
    print(result)

538703333547789


## Day 14: Bitmask

In [78]:
from typing import Union
import collections

def day14_parse(content: str) -> list[Union[str, tuple[int, int]]]:
    """Returns (address, value). 
    When the command is mask, address -1 is used."""
    cmd_value_pairs = [line.strip().split(" = ") for line in content.split("\n") if line.strip()]
    result = []
    for cmd, value_str in cmd_value_pairs:
        if cmd.startswith("mask"):
            result.append(value_str)
        else:
            assert cmd.startswith("mem")
            address = int(cmd[4:-1])
            value = int(value_str)
            result.append((address, value))
    return result


def day14_part1_apply(mask: str, x: int) -> int:
    """
    >>> day14_part1_apply("XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X", 0)
    64
    """
    for i, c in enumerate(reversed(mask)):
        if c != "X":
            b = list(bin(x)[2:].zfill(36))
            b[-i-1] = c
            x = int("".join(b), 2)
    return x

def day14_part1(commands: list[Union[str, tuple[int, int]]]) -> int:
    memory = collections.defaultdict(int)
    mask = "X" * 36
    for item in commands:
        if isinstance(item, str):
            mask = item
        else:
            addr, x = item
            memory[addr] = day14_part1_apply(mask, x)
    
    return sum(memory.values())


In [79]:
content = """
mask = XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X
mem[8] = 11
mem[7] = 101
mem[8] = 0
"""

commands = day14_parse(content)
day14_part1(commands)

165

In [80]:
with open("2020/day14_input.txt") as f:
    commands = day14_parse(f.read())
    res = day14_part1(commands)
    print(res)

11179633149677


In [81]:
from typing import Generator

def day14_part2_apply(mask: str, addr: int) -> list[int]:
    """
    >>> day14_part2_apply("000000000000000000000000000000X1001X", 42)
    [26, 27, 58, 59]

    >>> day14_part2_apply("00000000000000000000000000000000X0XX", 26)
    [16, 17, 18, 19, 24, 25, 26, 27]
    """
    s = list(bin(addr)[2:].zfill(36))

    for i, c in enumerate(reversed(mask)):
        if c == 'X':
            s[-i-1] = c
        else:
            bit1 = int(c)
            bit2 = int(s[-i-1])
            s[-i-1] = str(bit1 | bit2)

    res = [0]
    for i, c in enumerate(reversed(s)):
        if c == "X":
            res.extend([n + (1 << i) for n in res])
        elif c == '1':
            res = [n + (1 << i) for n in res]

    return res

def day14_part2(commands: list[Union[str, tuple[int, int]]]) -> int:
    memory = collections.defaultdict(int)
    mask = "X" * 36
    for item in commands:
        if isinstance(item, str):
            mask = item
        else:
            addr, x = item
            for addr in day14_part2_apply(mask, addr):
                memory[addr] = x
    
    return sum(memory.values())


In [82]:
content = """
mask = 000000000000000000000000000000X1001X
mem[42] = 100
mask = 00000000000000000000000000000000X0XX
mem[26] = 1
"""

commands = day14_parse(content)
day14_part2(commands)


208

In [83]:
with open("2020/day14_input.txt") as f:
    commands = day14_parse(f.read())
    res = day14_part2(commands)
    print(res)

4822600194774


## Day 15: Sequence

**[FIXME]** My part 2 solution takes over 10 seconds with Intel i7-3520M, which is not cool.

In [88]:
from typing import Generator
import itertools

def day15_part1_gen(seq: list[int]) -> Generator[int, None, None]:
    if seq:
        yield from seq
    else:
        seq.append(0)
        yield 0

    while True:    
        xlast = seq[-1]
        indices = [i for i, x in enumerate(seq) if x == xlast]
        if len(indices) == 1:
            x_next = 0
        else:
            ia, ib = indices[-2:]
            x_next = ib - ia
        
        seq.append(x_next)
        yield x_next

def day15_part1(seq: list[int], pos: int=2020):
    res = next(itertools.islice(day15_part1_gen(seq), pos - 1, pos))
    return res

In [89]:
seq = [0, 3, 6]
list(itertools.islice(day15_part1_gen(seq), 10))

[0, 3, 6, 0, 3, 3, 1, 0, 4, 0]

In [90]:
seq = [0, 3, 6]
day15_part1(seq)

436

In [91]:
seq = [2,15,0,9,1,20]
day15_part1(seq)

1280

In [92]:

def day15_part2_gen(seq: list[int]) -> Generator[int, None, None]:
    t = 1
    last_call: dict[int, int] = dict()
    assert seq

    for x in seq[:-1]:
        yield x
        last_call[x] = t
        t += 1
    
    x_last = seq[-2]
    x = seq[-1]

    while True:
        yield x
        x_last = x
        x = t - last_call[x] if x in last_call else 0

        last_call[x_last] = t
        t += 1


def day15_part2(seq: list[int], pos: int=30000000):
    res = next(itertools.islice(day15_part2_gen(seq), pos - 1, pos))
    return res

In [93]:
seq = [0, 3, 6]
list(itertools.islice(day15_part2_gen(seq), 10))

[0, 3, 6, 0, 3, 3, 1, 0, 4, 0]

In [94]:
seq = [0, 3, 6]
day15_part2(seq)

175594

In [95]:
seq = [2,15,0,9,1,20]
day15_part2(seq)

651639